In [2]:
# Importing all necessary libraries
import os
import pandas as pd  # For handling data in tables (DataFrames)
import SimpleITK as sitk
from radiomics import featureextractor  
# The main PyRadiomics class
import logging

# --- Configuration ---
# Setting up a logger to show PyRadiomics progress and warnings, but not too much detail

logging.getLogger('radiomics').setLevel(logging.WARNING)

# Setting the base path to our data
# This is where the images and masks are stored
data_base_path = '../Data/HCC-TACE-Seg'

print("Libraries imported and configuration set.")

Libraries imported and configuration set.


In [3]:
# --- Extractor Setup ---
# Initializing the feature extractor
extractor  = featureextractor.RadiomicsFeatureExtractor()   # This is the default extractor 

#Now manuaally setting up the extractor parameters      
#This is the modern way to set up the extractor, which allows for more customization
print("Custt")
# Telling PyRadiomics what features to extract
# doing this by using a special parameter file. Creating a basic setup in code
params = {}
params['binWidth'] = 25 #standard setting for CT scans
params['resampledPixelSpacing'] = [1,1,1] #Resampling all images to 1x1x1 mm voxels for consistency
params['interpolator'] = sitk.sitkBSpline # Using B-spline interpolation for resampling
params['label'] = 255 #telling it to only look at the tumor label we found earlier

#Initializing the feature extractor with the setting

try:
    extractor = featureextractor.RadiomicsFeatureExtractor(params)
except Exception as e:
    #if the above fails because of an old pyradiomics version, this might work
    print(f"Initial extractor setup failed with error: {e}. Trying alternative setup.")
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()  # a less practice but robust method

print("PyRadiomics feature exttractor initialized.")

validation.invalid
 --- All found errors ---
["Key 'binWidth' was not defined. Path: ''", "Key 'resampledPixelSpacing' was not defined. Path: ''", "Key 'interpolator' was not defined. Path: ''", "Key 'label' was not defined. Path: ''"]


Custt
Initial extractor setup failed with error: <SchemaError: error code 2: Schema validation failed:
 - Key 'binWidth' was not defined. Path: ''.
 - Key 'resampledPixelSpacing' was not defined. Path: ''.
 - Key 'interpolator' was not defined. Path: ''.
 - Key 'label' was not defined. Path: ''.: Path: '/'>. Trying alternative setup.
PyRadiomics feature exttractor initialized.


In [4]:
# --- Main Loop ---

# This will store the results for all patients
all_features = []

# Geting a sorted list of all patient folders
patient_folders = sorted([f for f in os.listdir(data_base_path) if os.path.isdir(os.path.join(data_base_path, f))])

print(f"Found {len(patient_folders)} patients. Starting extraction...")

# Looping through each patient folder

for patient_id in patient_folders:
    print(f"Processing patient: {patient_id}...")
    patient_path = os.path.join(data_base_path, patient_id)
    
    # --- Step 1: Find Image and Mask files (using our robust logic from Task 03) ---

    image_series_path = ''
    mask_file_path = ''

    # Walking through all the directories and files inside the patient folder
    # to find the image series and mask file
    for root, dirs, files in os.walk(patient_path):
        if 'SEGMENTATION' in root.upper():
            # If I find a segmentation folder, I assume it contains the mask
            if files: mask_file_path = os.path.join(root, files[0])

        if len(files) > 50 and files[0].endswith('.dcm'):
            # If I find a folder with more than 50 DICOM files, I assume it's the image series
            image_series_path = root

    # --- Step 2: Load and Resample (if paths were found) ---

    if image_series_path and mask_file_path:
        try:
            
            # Load image
            image_reader = sitk.ImageSeriesReader()
            dicom_names = image_reader.GetGDCMSeriesFileNames(image_series_path)
            image_reader.SetFileNames(dicom_names)
            image_3d = image_reader.Execute()

            # Load mask
            mask_3d = sitk.ReadImage(mask_file_path)

            # Resample mask to match image
            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(image_3d)
            resampler.SetInterpolator(sitk.sitkNearestNeighbor)
            resampled_mask_3d = resampler.Execute(mask_3d)

            # --- Step 3: Extract Features ---
            # The extractor takes the image and the resampled mask
            # It's crucial to use the original image and the resampled mask, not the original mask
            feature_vector = extractor.execute(image_3d, resampled_mask_3d, label=255)
            
            # Add the patient ID to our results
            feature_vector['PatientID'] = patient_id
            all_features.append(feature_vector)
            
            print(f"  > Success! Extracted {len(feature_vector)-1} features.")

        except Exception as e:
            print(f"  > FAILED for patient {patient_id}. Error: {e}")
    else:
        print(f"  > FAILED for patient {patient_id}. Could not find image or mask path.")

print("\nFeature extraction complete.")

Found 105 patients. Starting extraction...
Processing patient: HCC_001...


  > FAILED for patient HCC_001. Error: No labels found in this mask (i.e. nothing is segmented)!
Processing patient: HCC_002...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 129 features.
Processing patient: HCC_003...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...

  > Success! Extracted 129 features.
Processing patient: HCC_104...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 129 features.
Processing patient: HCC_105...


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  > Success! Extracted 129 features.

Feature extraction complete.


In [5]:
# --- Saving Results ---

# Convertint the list of feature dictionaries into a pandas DataFrame
# A DataFrame is like a spreadsheet or a table

features_df = pd.DataFrame(all_features)

# Setting the order of the columns to get the PatientID as the first column for easy reference

cols = ['PatientID'] + [col for col in features_df.columns if col != 'PatientID']

features_df = features_df[cols]

# Setting the output path for the output file
output_path = '../Results/radiomics_features.csv'

# Saving the DataFrame to a CSV file
features_df.to_csv(output_path, index=False)

print(f"Successfully saved {len(features_df)} patient records to:")
print(os.path.abspath(output_path))

# Displaying the first 5 rows of the new table
features_df.head()

Successfully saved 98 patient records to:
c:\Thesis_Project\Results\radiomics_features.csv


,PatientID,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,HCC_002,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},fd7992bb639fb9faf665ba819bbd34498377504f,3D,...,1099.8149208591517,0.0023697801381916167,7.583554384864127,0.26679577935087484,2054.359117482585,0.2396853067339907,0.0008502342467176705,5433.018956955054,0.06080326794438817,5.615451453425239
1,HCC_003,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},25aeb8d1ec7731ede2658379e04e15b3befd6497,3D,...,769.3533178996998,0.023946351412725417,7.485852328292948,0.12228727025187203,4584.584626934664,0.7698898280558546,0.00045715253239660977,3697.1129554536174,0.26981151141155885,2.4314885704488103
2,HCC_004,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3b109c7a737933c2ffe0dd9f7ff6ba61aa1617af,3D,...,982.9051042609103,0.005597864447674355,8.184989238560847,0.1478127217304788,71788.31617227747,6.842687288463462,3.058283611455279e-05,18299.8578155048,0.0668671241145693,0.6895097409876039
3,HCC_005,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},b3321933601c857f30931e33f848083b44d67fbe,3D,...,1077.3630471870777,0.0011338962311964428,7.141550907621083,0.11574919622918968,222863.08800993662,10.955859892357402,1.6832631818601294e-05,3817.8404985062903,0.015906301239814284,0.08767486633063029
4,HCC_006,v3.0.1,2.2.6,2.5.2,1.8.0,3.10.16,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},a0a1238368bdb1bbc105ea0be520016c306efe99,3D,...,1291.330916101148,0.0028918655074671516,7.605537387592789,0.12340219491115088,78501.19701343217,6.983019626887953,2.1857447738666883e-05,5438.366683688181,0.028211533424528937,0.18943655461842052
